In [ ]:
from init_notebook import *

In [ ]:
PATH = Path("../../image-scraper/export/")
features = []
meta_data = []
for i in tqdm(range(15)):
    features.append(np.load(PATH / f"export_imagefap_{i:04}_features.npy"))
    meta_data.extend(json.loads((PATH / f"export_imagefap_{i:04}_meta.json").read_text()))
features = np.concatenate(features)
for i, data in enumerate(meta_data):
    data["index"] = i
features.shape

In [ ]:
features_norm = features / np.linalg.norm(features, axis=-1, keepdims=True)

In [ ]:
px.line(features_norm[:3].T)

In [ ]:
ratings = {}
for data in meta_data:
    if data["rating"][0]["user"]:
        ratings[data["index"]] = data["rating"][0]["rate"]
len(ratings)

In [ ]:
from IPython.display import HTML

BASE_URL = "http://5.9.84.201/isf/"
def html_image(*index: int):
    markup = []
    for i in index:
        img_url = BASE_URL + meta_data[i]["thumb_filename"][9:]
        url = meta_data[i]["url"]
        markup.append(f"""<div><a href="{url}"><img src="{img_url}"></div>""")

    markup = "".join(markup)
    return HTML(f"""<div style="display: flex; flex-wrap: wrap;">{markup}</div>""")

#html_image(4)
html_image(*(i for i, r in ratings.items() if r > 0))

In [ ]:
from sklearn.linear_model import Ridge
#from sklearn.gaussian_process import GaussianProcessClassifier

NUM_RANDOM = 0
random_features = []
for i in range(NUM_RANDOM):
    while True:
        i = random.randrange(features.shape[0])
        if i not in ratings:
            random_features.append(features[i])
            break
            
rated_features = np.stack([features[i] for i in ratings] + random_features)
rated_targets = np.array(list(ratings.values()) + [0] * NUM_RANDOM)
print(rated_features.shape, rated_targets.shape)

classifier = Ridge()
#classifier = GaussianProcessClassifier()
classifier.fit(rated_features, rated_targets)
predicted_ratings = classifier.predict(features)
predicted_ratings.shape

In [ ]:
top_indices = predicted_ratings.argsort()[::-1]
html_image(*top_indices[:100])

In [ ]:
html_image(*top_indices[200:][:1000])

In [ ]:
positive_features = features_norm[[i for i, v in ratings.items() if v >= 1]]
positive_features.shape
mean_positive_feature = positive_features.mean(axis=0)
print(mean_positive_feature.shape)
px.line(positive_features[:5].T)

In [ ]:
sim = features_norm @ positive_features.T
sim.shape

In [ ]:
sim_m = mean_positive_feature @ features_norm.T 
sim_m.shape

In [ ]:
top_indices = sim_m.argsort()[::-1]
html_image(*top_indices[:1000])